In [2]:
# !pip unistall ipywidgets
# !pip install ipywidgets
# !pip install tabulate
# !pip install plotly 

%run lab5.py

Tab(children=(VBox(children=(HTML(value="<div align='center'><font size='6px' face='Arial' color='#0969A2'>Lab 5</font></div>"), HTML(value="<div align='center'><font size='5px' face='Arial' color='#0969A2'>Дрон. Требования к функциональным элементам</font></div>"), HBox(children=(VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Внешний вид</font></div>"), Dropdown(description='Материал корпуса', layout=Layout(justify_content='center', width='400px'), options=('Волоконный композит', 'Кевлар', 'Углепластика', 'Экструдированный пенополистирол'), style=DescriptionStyle(description_width='initial'), value='Волоконный композит'), Checkbox(value=False, description='Портативность', style=DescriptionStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Механика</font></div>"), Dropdown(description='Тип двигателя', layout=Layout(justify_content='center', width='400px'), options=('Бесколлекторный электродвигатель', 'Коллекторный электродвигатель', 'Электродвигатель'), style=DescriptionStyle(description_width='initial'), value='Бесколлекторный электродвигатель'), FloatRangeSlider(value=(2.0, 6.0), continuous_update=False, description='Число двигателей', layout=Layout(justify_content='flex-start', width='400px'), max=6.0, min=2.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial')), Dropdown(description='Тип движителя', layout=Layout(justify_content='center', width='400px'), options=('Винт', 'Реактивная тяга'), style=DescriptionStyle(description_width='initial'), value='Винт'), Dropdown(description='Материал пропеллера', layout=Layout(justify_content='center', width='400px'), options=('Волоконный композит', 'Карбон'), style=DescriptionStyle(description_width='initial'), value='Волоконный композит'), FloatRangeSlider(value=(2.0, 6.0), continuous_update=False, description='Число пропеллеров', layout=Layout(justify_content='flex-start', width='400px'), max=6.0, min=2.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'></font></div>"), FloatRangeSlider(value=(15000.0, 30000.0), continuous_update=False, description='Стоимость, грн', layout=Layout(justify_content='flex-start', width='400px'), max=30000.0, min=15000.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial')), FloatRangeSlider(value=(800.0, 1000.0), continuous_update=False, description='Общий вес, г', layout=Layout(justify_content='flex-start', width='400px'), max=1000.0, min=800.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='10px', width='480px')), Button(description='Анализ', style=ButtonStyle())), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px')), VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Аппаратная составляющая</font></div>"), Checkbox(value=False, description='Наличие GPS', style=DescriptionStyle(description_width='initial')), Checkbox(value=False, description='Наличие ИИ', style=DescriptionStyle(description_width='initial')), Dropdown(description='Производитель ', layout=Layout(justify_content='center', width='400px'), options=('Agisoft', 'Harris Geospatial Solutions', 'Trimble'), style=DescriptionStyle(description_width='initial'), value='Agisoft'), FloatRangeSlider(value=(1.6, 3.0), continuous_update=False, description='Частота процессора, Гц', layout=Layout(justify_content='f

[{'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Luftera', 'price': 1400, 'weight': 200, 'duration': 360, 'take_off_weight': 5, 'height': 150}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'TMS320C6678', 'price': 460, 'weight': 0, 'frequency': 1.8, 'num_cores': 6}, 'Propeller': {'name': 'Phantom 3', 'price': 200, 'weight': 30, 'material': 'Волоконный композит', 'number': 2}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT 6600AUXSDWF', 'price': 1750, 'weight': 200, 'resolution': 2.4, 'angle': 180, 'IR_illumination': 35}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight'

[{'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight':

In [3]:
import pandas as pd

In [4]:
l = [{'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}, 'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'Phantom 5', 'price': 245, 'weight': 50, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'AVT AHDCH 108', 'price': 1200, 'weight': 200, 'resolution': 2, 'angle': 180, 'IR_illumination': 18}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}, {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}, 'Accumulator': {'name': 'Skywalker X8', 'price': 2100, 'weight': 200, 'duration': 240, 'take_off_weight': 8, 'height': 1200}, 'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}, 'CPU': {'name': 'DSP А871', 'price': 750, 'weight': 0, 'frequency': 3, 'num_cores': 8}, 'Propeller': {'name': 'AEE N7638', 'price': 240, 'weight': 75, 'material': 'Волоконный композит', 'number': 4}, 'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 'propulsion_type': 'Винт'}, 'Camera': {'name': 'JIMI JH012', 'price': 980, 'weight': 200, 'resolution': 2.4, 'angle': 105, 'IR_illumination': 15}, 'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 'portability': True, 'material': 'Волоконный композит'}}]

In [30]:
# for key in l[0].keys():
#     print(l[0][key])
#     pd.DataFrame.from_dict(l[0][key])

# pd.DataFrame.from_dict([l_i['Body'] for l_i in l])

In [12]:
table = pd.DataFrame.from_dict(l[0])
table[table.isna()] = '-'
table

,Accumulator,Body,CPU,Camera,Engine,Propeller,Sensor,Software
AI,-,-,-,-,-,-,-,False
GPS,-,-,-,-,-,-,-,True
IR_illumination,-,-,-,18,-,-,-,-
angle,-,-,-,180,-,-,-,-
duration,240,-,-,-,-,-,-,-
engine_type,-,-,-,-,Бесколлекторный электродвигатель,-,-,-
frequency,-,-,2.7,-,-,-,-,-
height,1200,-,-,-,-,-,-,-
manufacturer,-,-,-,-,-,-,-,Agisoft
material,-,Волоконный композит,-,-,-,Волоконный композит,-,-


In [8]:
l[0]

{'Accumulator': {'duration': 240,
  'height': 1200,
  'name': 'Skywalker X8',
  'price': 2100,
  'take_off_weight': 8,
  'weight': 200},
 'Body': {'material': 'Волоконный композит',
  'name': 'AgDrone',
  'portability': True,
  'price': 15000,
  'weight': 180},
 'CPU': {'frequency': 2.7,
  'name': 'DSP C66x',
  'num_cores': 8,
  'price': 980,
  'weight': 0},
 'Camera': {'IR_illumination': 18,
  'angle': 180,
  'name': 'AVT AHDCH 108',
  'price': 1200,
  'resolution': 2,
  'weight': 200},
 'Engine': {'engine_type': 'Бесколлекторный электродвигатель',
  'name': 'Syma NHYDXS',
  'number': 2,
  'price': 1600,
  'propulsion_type': 'Винт',
  'weight': 330},
 'Propeller': {'material': 'Волоконный композит',
  'name': 'Phantom 5',
  'number': 4,
  'price': 245,
  'weight': 50},
 'Sensor': {'name': 'Namur',
  'price': 0,
  'signal_acceptance_distance': 1,
  'weight': 0},
 'Software': {'AI': False,
  'GPS': True,
  'manufacturer': 'Agisoft',
  'name': 'Agisoft Photoscan',
  'price': 360,
  'weig